# COVID-19 Analysis Platform

In [30]:
from jupyter_dash import JupyterDash

In [31]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd

Load and preprocess data

In [32]:
df = pd.read_csv("owid-covid-data.csv")

In [33]:
countries = df.location.unique()

Construct the app and callbacks

In [43]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

app.layout = html.Div([
    html.Div([

        html.Div(
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in countries],
                value='Norway'
            ),
        style={'width': '49%', 'display': 'inline-block'}),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'})
])
])


@app.callback(
    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
    dash.dependencies.Input('crossfilter-xaxis-column', 'value'))
def update_graph(xaxis_column_name):
    dff = df[df.location == xaxis_column_name]

    return {
        'data': [dict(
            x=dff.date,
            y=dff.new_cases,
            text=xaxis_column_name,
            customdata='Test',
            type='bar'
#             mode='markers',
#             marker={
#                 'size': 25,
#                 'opacity': 0.7,
#                 'color': 'orange',
#                 'line': {'width': 2, 'color': 'purple'}
#             }
        )],
        'layout': dict(
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }




Serve the app using `run_server`.  Unlike the standard `Dash.run_server` method, the `JupyterDash.run_server` method doesn't block execution of the notebook. It serves the app in a background thread, making it possible to run other notebook calculations while the app is running.

This makes it possible to iterativly update the app without rerunning the potentially expensive data processing steps.

In [45]:
app.run_server()

Dash app running on http://127.0.0.1:8050/


In [21]:
print(app._terminate_server_for_port(host='127.0.0.1',port='8050'))

None


By default, `run_server` displays a URL that you can click on to open the app in a browser tab. The `mode` argument to `run_server` can be used to change this behavior.  Setting `mode="inline"` will display the app directly in the notebook output cell.

In [7]:
app.run_server(mode="inline")

When running in JupyterLab, with the `jupyterlab-dash` extension, setting `mode="jupyterlab"` will open the app in a tab in JupyterLab.

```python
app.run_server(mode="jupyterlab")
```